In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import os
import sys
import cv2
import h5py
import math
import time
import serial
import random
import threading
import datetime
import numpy as np
from numpy import fft
from numpy import array
from sklearn import svm
from pickle import BINSTRING
from collections import deque
from sklearn import grid_search
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
#Object dectection Yolo CNN
sys.path.insert(0, '/home/rahulsingh/Desktop/Intention-Recognition/yolo')
from darkflow.net.build import TFNet
from sklearn import datasets
iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split

import pylab as pl
from IPython import display
from scipy.stats import moment
##########################################################################################################################
#Step by step debugging in iPython: https://stackoverflow.com/questions/16867347/step-by-step-debugging-with-ipython
#Multi-threading in python: https://stackoverflow.com/questions/31768865/how-to-use-threading-to-get-user-input-realtime-while-main-still-running-in-pyth
##########################################################################################################################

objList = ['apple', 'cup', 'bottle', 'keyboard', 'cell phone']
objCount= np.array((0, 0, 0, 0, 0))
imu_num = 2
imu_data_chunk = 100
inp = '3'

###Selectively write data
#And threshold values for each
#Convert counter into probability score
#save intent data in h5py file
#fix the timing issue in saving the data
#Is normalization of data required??
#What other features more can be added further to increase accuracy??
#What more extra information needs to be added (for false positive) when hands takes away from the object?
#Load||collect data
##data = collectData(person='rahul', port0='0', port1='1')
##Pre-process data
##trainX, trainY = preProcess(data)

In [ ]:
#Create a file in 'h5py' file and save list continously when called
#Arguments: filename (string)
##Check the storing of "TIME STAMP" data ==> there is some problem with it
def createFile(filename):
    flist=[]
    try:
        file = h5py.File(filename, "w")
    except:
        print("Error: Creating dataset file")
        file.close()

    for i in range(imu_num):
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/acc", (0,3), maxshape=(None,3), dtype='i'))
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/gyro",(0,3), maxshape=(None,3), dtype='i'))
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/mag", (0,3), maxshape=(None,3), dtype='i'))
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/time",(0,1), maxshape=(None,1), dtype='float64'))
        flist.append(file.create_dataset("mocap/imu{}".format(i) + "/intent", (0,1), maxshape=(None,1), dtype='int8'))
        
    flist.append(file.create_dataset("camera/image", (0, 240, 320, 3), maxshape=(None, 240, 320, 3), dtype='uint8'))
    flist.append(file.create_dataset("camera/time",(0,1), maxshape=(None,1), dtype='float64'))
    flist.append(file.create_dataset("camera/intent", (0,1),maxshape=(None,1), dtype='int8'))

    return file, flist

def saveData(flist, connection, imu_data, tstamp, intent):
    data = np.array([np.array(xi) for xi in imu_data[connection]])#, 'int8')#.T #take care of unsigned to signed int conversion
    cur_size = len(flist[connection])       #current row size of h5 file
    data_size= len(imu_data[connection])    #incoming data row size
    
    for i in range(3):
        flist[connection*5 + i].resize((cur_size + data_size, 3))                         #extending h5 row size
        flist[connection*5 + i][cur_size: cur_size + data_size] = data[:, 3*i:3*i+3]      #data[3*i : 3*i + 3].T   #writing imu data to corresponding subgroup
        
    time = np.array(tstamp[connection], dtype=np.float32)                                 #converting time stamp data to numpy array (float)
    flist[connection*5 + 3].resize((cur_size + data_size, 1))                             #resizing the file dataset for storing
    flist[connection*5 + 3][cur_size: cur_size + data_size] = time.reshape((data_size,1)) #finally storing time stamp data
    
    intent=np.array(intent[connection], dtype='int8') 
    flist[connection*5 + 4].resize((cur_size + data_size, 1))                             #resizing the file dataset for storing
    flist[connection*5 + 4][cur_size: cur_size + data_size] = intent.reshape((data_size,1))
    return 0

def loadData(d):
    return 0

def closeFile(file):
    file.close()
    return 0
    
def getIMUData(flist, person='test', port0 = '0', port1= '1'):
    global inp, imu_data_chunk, imu_num
    
    imu_data  = deque([deque([]) for _ in range(imu_num)])
    tstamp    = deque([deque([]) for _ in range(imu_num)])
    intent    = deque([deque([]) for _ in range(imu_num)])

    ser0 = serial.Serial('/dev/ttyACM' + port0, 115200) # Establish the connection on a specific port
    ser1 = serial.Serial('/dev/ttyACM' + port1, 115200) # Establish the connection on a specific port
    now      = datetime.datetime.now()
    fhandle  = open("data/" + person + "_" + now.isoformat() + "_imu.txt", 'ab')
    imu      = []  #contains tstamps with imu data
    ser0.flush()
    ser1.flush()
    while inp!='s':
        data0 = ser0.readline().rstrip()
        data0 = data0.split()
        data1 = ser1.readline().rstrip()
        data1 = data1.split()
        if ((len(data0) == 6) & (len(data1) == 6) & (inp<'3')):
            tstamp[0].append(time.time())
            tstamp[1].append(time.time())
            intent[0].append(int(inp))
            intent[1].append(int(inp))
            imu_data[0].append([int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), 0, 0, 0])
            imu_data[1].append([int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5]), 0, 0, 0])
            imu.append([float(time.time()), int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5]), int(inp)])
            #imu.append([float(time.time()), inp, (int(data0[0])*2.0)/ 32768.0, (int(data0[1])*2.0)/ 32768.0, (int(data0[2])*2.0)/ 32768.0, (int(data0[3])*250.0)/32768.0, (int(data0[4])*250.0)/32768.0, (int(data0[5])*250.0)/32768.0, (int(data1[0])*2.0)/ 32768.0, (int(data1[1])*2.0)/ 32768.0, (int(data1[2])*2.0)/ 32768.0, (int(data1[3])*250.0)/32768.0, (int(data1[4])*250.0)/32768.0, (int(data1[5])*250.0)/32768.0])
        
        if((len(imu_data[0]) == imu_data_chunk) or (inp == 's') & (len(imu_data[0]) != 0)):
            saveData(flist, 1, imu_data, tstamp, intent)
            saveData(flist, 0, imu_data, tstamp, intent)
            print("Average Sampling Freq. of", len(imu_data[0]), "elements is ::", getFreq(tstamp, 0, show=False))     #imu_data_chunk/(tstamp[connection][0]-tstamp[connection][299])
            print("Average Sampling Freq. of", len(imu_data[0]), "elements is ::", getFreq(tstamp, 1, show=False))
            imu_data[0].clear()
            tstamp[0].clear()     #frequency = imu_data_chunk/(tstamp[connection][0]-tstamp[connection][299])
            imu_data[1].clear()
            tstamp[1].clear()
            intent[0].clear()
            intent[1].clear()

    np.savetxt(fhandle, np.array(imu), delimiter=",")
    fhandle.close()
    ser0.close()
    ser1.close()
        
    return imu

#Using thread to store the user input for task classification
def background(flist):
    #global imu_num, inp, camera, image
    camera = cv2.VideoCapture(1)
    camera.set(3,320)
    camera.set(4,240)  
    print("Camera FPS:: ", camera.get(cv2.CAP_PROP_FPS))
    while inp!='s':
        tst = time.time()
        ret, image = camera.read()
        cv2.imshow('frame', image)
        if(cv2.waitKey(1) & 0xFF == ord('q')):
            break
        tsp=time.time()
        if(inp<'3'):      
            cur_size = len(flist[imu_num*5])                     #current row size of h5 file
            flist[imu_num*5].resize((cur_size + 1, 240, 320, 3)) #resizing the file dataset for storing
            flist[imu_num*5][cur_size: cur_size + 1] = image
            flist[imu_num*5 + 1].resize((cur_size + 1, 1))       #resizing the file dataset for storing
            flist[imu_num*5 + 1][cur_size: cur_size + 1] = tst
            flist[imu_num*5 + 2].resize((cur_size + 1, 1))       #resizing the file dataset for storing
            flist[imu_num*5 + 2][cur_size: cur_size + 1] = int(inp)
        
    camera.release()
    cv2.destroyAllWindows()
    return 0

def getRefInput():
    global inp         #Allow only Legitimate Inputs
    while inp!='s':
        inp = input()
        #time.sleep(0.001)
        #inp = int(input())
    return 0

def getFreq(tstamp, connection, show=False):
    global imu_data_chunk
    tm = np.array(tstamp[connection])
    dt = tm[1:len(tm)-1] - tm[0:len(tm)-2]
    freq = 1./dt
    if show:
        print("Individual Sampling Freq.:: ", freq,"\n")
    avg_freq = len(dt)/np.sum(dt)   #getting average frequency
    return avg_freq

In [ ]:
def detectObject(inp_image, result):    #Check for VALID image first
    objFlag = np.array((0, 0, 0, 0, 0))
    out = (tfnet.return_predict(inp_image))
    result.append(out)
    for obj in out:
        cv2.rectangle(inp_image, (obj["topleft"]["x"], obj["topleft"]["y"]), (obj["bottomright"]["x"], obj["bottomright"]["y"]), (0, 255, 0), 4)
        text_x, text_y = obj["topleft"]["x"] - 10, obj["topleft"]["y"] - 10
        cv2.putText(inp_image, obj["label"], (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
        for i, j in enumerate(objList):
            if j == obj["label"]:
                objFlag[i] = 1

    return objFlag

def printObject(objFlag):
    obj = ""
    obj_index = np.where(objFlag == 1)[0]
    for i in range(len(obj_index)):        #print(objList[obj_index[i]])
        obj = objList[obj_index[i]] + ", " + obj
        
    print(obj)
    return 0

def loadWeight(model="gaussian"):
    options = {"model": "cfg/yolo.cfg", "load": "yolo/bin/yolo.weights", "threshold": 0.2}
    tfnet = TFNet(options)
    clf = joblib.load("weights/" + model + "_1s_6sps.pkl")
    print(clf)
    return tfnet, clf

In [ ]:
def getDataset(tstamps, imu_data, intent, stride=2, sigLen = 25, isfeature=True):
    train_x = []
    train_y = []  
    length = len(imu_data)
    if len(imu_data) < len(imu_data.T):
        length   = len(imu_data.T)
        imu_data = imu_data.T
        
    if len(intent)!=len(imu_data):
        print("Input dimension of trainX & trainY different! Check!")
        return None
    
    #Make useful for multi-class classification
    for start in  range(0, length-sigLen, stride):
        if ((sum(intent[start: start + stride])<stride) & (sum(intent[start: start + stride])>0)): #((clas[start: start + stride]==).all()):
            continue
        if ((start + sigLen) <= length):
            train_x.append(imu_data[start:start+sigLen])
            train_y.append(intent[start])
    
    if not isfeature: return train_x, train_y
    #Now extract feature from each signal stamp
    trainX = []
    trainY = np.array(train_y).reshape((len(train_y,)))
    train_data = np.array(train_x)
    
    for i in range(len(train_x)):
        #trainX.append(np.array((np.array(moment(train_data[i], moment=4)), 
        #                        np.array(moment(train_data[i], moment=3)), 
        #                        np.array(moment(train_data[i], moment=2)), 
        #                        mean(train_data[i]))).reshape((24*2,)))
        trainX.append(np.array((stdev(train_data[i]), mean(train_data[i]))).reshape((24,)))
        
    return trainX, trainY

def trainModel(trainX, trainY, func='gaussian'):#change name of this function
    if func=='bernoulli':     clf = BernoulliNB()
    elif func=='multinomial': clf = MultinomialNB()
    elif func=='svm':
        svr   = svm.SVC()
        exponential_range = [pow(10, i) for i in range(-4, 1)]
        parameters = {'kernel':['linear', 'rbf'], 'C':exponential_range, 'gamma':exponential_range}
        clf = grid_search.GridSearchCV(svr, parameters, n_jobs=8, verbose=True)
    else: clf = GaussianNB()
    
    X_train, X_test, Y_train, Y_test = train_test_split(trainX, trainY, test_size=0.30, random_state=0)
    clf.fit(X_train, Y_train)
    
    print("Saving the model...",)
    joblib.dump(clf, 'weights/' + func + '_1s_6sps.pkl')
    
    print("Confusion Matrix:")
    y_pred = clf.predict(np.array(X_test))
    print(confusion_matrix(Y_test, y_pred))

    #Calculates the score of the best estimator found.
    score = clf.score(X_test, Y_test)
    print("\nSCORE: {score}\n".format(score = score))
    print("DONE")
    #print("Number of mislabeled points out of a total %d points : %d" % (np.array(X_test).shape[0],(Y_test != y_pred).sum()))
    #print("Accuracy Percentage", (1-(((Y_test != y_pred).sum())/(np.array(X_test).shape[0])))*100)    
    return clf, score

def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return np.sqrt(variance)

def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [ ]:
def getCamScore(flag, del_time, prev_score, beta = 1.0):
    score = []
    prev_score = np.array(prev_score)
    flag = np.array(flag)
    for i in range(5):
        if ((flag[i]==1) & (prev_score[i] < 1.0)):
            score.append(prev_score[i] + del_time*beta)
        elif ((flag[i]==0) & (prev_score[i] > 0.0)):
              score.append(prev_score[i] - del_time*beta)
        else:
              score.append(prev_score[i])
    #score = (1+np.tanh(del_time - 2))/2
    return score

def getIMUScore(flag, del_time, prev_score, beta=1.0):
    score = []
    flag = int(flag)
    if ((flag==1) & (prev_score < 1.0)):
        score = prev_score + del_time*beta
    elif ((flag==0) & (prev_score > 0.0)):
          score = prev_score - del_time*beta
    else:
          score = prev_score
    return score

def totScore(cam_score, imu_score):
    return np.array(cam_score)*imu_score

def detectIntent(cam_score, imu_score, threshold, objList):
    totscore = totScore(cam_score, imu_score)
    for i in range(5):
        if totscore[i]>threshold:
            print("Intention is probably to pickup:", objList[i])
            totscore[i] = 0
            cam_score[i]= 0
            imu_score   = 0
            
    return totscore, cam_score, imu_score

def loadData(filename):
    return np.loadtxt("data/" + filename + ".txt", delimiter=",")

def storeData(filename = 'test', port1 = '0', port2 = '1'):
    now      = datetime.datetime.now()
    file, flist = createFile('data/' + filename + '_' +  now.isoformat() + '.h5')
    thread1 = threading.Thread(target=background, args = [flist])
    thread1.daemon = True
    thread1.start()
    thread2 = threading.Thread(target=getRefInput)
    thread2.daemon = True
    thread2.start()
    thread3 = threading.Thread(target=getIMUData, args=[flist, filename, port1, port2])
    thread3.daemon = True
    thread3.start()
    while(inp!='s'):
        time.sleep(0.01)
        
    file.close()
    #dat = getIMUData(flist, filename, port1, port2)
    #If want to train from imu data saved in file
    isTrain = False
    if isTrain:
        trainModel(trainX, trainY)
    
    return 0

#Basic plotting function of IMUs data, currently IMU is plotted
def plot(acc0, gyro0, acc1, gyro1, tstamp, x, y, z):
    #display.clear_output(wait=True)
    f, axarr = plt.subplots(3, imu_num, figsize=(15,12))
    axarr[0, 0].plot(acc0)
    axarr[0, 1].plot(gyro0)
    axarr[1, 0].plot(acc1)
    axarr[1, 1].plot(gyro1)
    axarr[2, 0].plot(x)
    axarr[2, 1].plot(y)
    axarr[2, 1].plot(z)
    axarr[2, 1].plot(0.8)    
    plt.show()
    display.display(pl.gcf())
    return 0

#storeData()

In [ ]:
inp = '3'
hf = h5py.File('data/rahul_april10.h5', 'r')

acc0  = hf.get('mocap/imu0/acc')
gyro0 = hf.get('mocap/imu0/gyro')
acc1  = hf.get('mocap/imu1/acc')
gyro1 = hf.get('mocap/imu1/gyro')
t_imu = hf.get('mocap/imu0/time')
in_imu= hf.get('mocap/imu0/intent')
in_cam= hf.get('camera/intent')
cam   = hf.get('camera/image')
t_cam = hf.get('camera/time')

##Just for temporary Testing##
tmp = loadData('rahul_april10')
t_imu = tmp.T[0]
xp = np.linspace(0, 1, len(t_imu))
fp = t_imu
yp = np.linspace(0, 1, len(cam))
t_cam = np.interp(yp, xp, fp)
in_imu = np.array(in_imu)
in_imu[in_imu==2] = 1

tfnet, clf = loadWeight()
result = []
display.clear_output()

thread3 = threading.Thread(target=getRefInput)
thread3.daemon = True
thread3.start()

tot_frame= len(cam) - 1
frameNum = 1560
cam_flag  = [0, 0, 0 , 0, 0]
cam_score = [0, 0 ,0 , 0, 0]
imu_flag  = 0
imu_score = 0
x = deque([])
y = deque([])
z = deque([])
while ((frameNum < tot_frame) & (inp!='s')):  #for i in range(len(cam)-1):
    inp_image = cam[frameNum]
    flag  = detectObject(inp_image, result)
    del_t = t_cam[frameNum + 1] - t_cam[frameNum]
    if del_t < 1.0:  
        cam_prev_score = cam_score
        cam_score = getCamScore(flag, del_t, cam_prev_score, beta = 1.2)
        
        imuFrameNum = int((len(acc0)/tot_frame)*frameNum)
        acc0Frame   = acc0[imuFrameNum-25:imuFrameNum]*2.0/32768.0
        gyro0Frame  = gyro0[imuFrameNum-25:imuFrameNum]*250.0/32768.0
        acc1Frame   = acc1[imuFrameNum-25:imuFrameNum]*2.0/32768.0
        gyro1Frame  = gyro1[imuFrameNum-25:imuFrameNum]*250.0/32768.0
        tIMUFrame   = t_imu[imuFrameNum-25:imuFrameNum]
        
        data = np.array([acc0Frame, gyro0Frame, acc1Frame, gyro1Frame]).reshape(25,12)
        inp  = np.array((stdev(data), mean(data))).reshape((1,24))
        out  = clf.predict(inp)
        imu_prev_score = imu_score
        imu_score = getIMUScore(int(out), del_t, imu_prev_score, beta = 2.0)   #del_t will be of imu NOT the camera one
        
        display.clear_output()
        plot(acc0[imuFrameNum-100:imuFrameNum],
             gyro0[imuFrameNum-100:imuFrameNum]*250.0/32768.0,
             acc1[imuFrameNum-100:imuFrameNum]*2.0/32768.0, 
             gyro1[imuFrameNum-100:imuFrameNum]*250.0/32768.0, 
             t_imu[imuFrameNum-100:imuFrameNum], x, y, z)
        tot_score, cam_score, imu_score = detectIntent(cam_score, imu_score, 0.7, objList)
        x.append(tot_score)
        y.append(cam_score)
        z.append(imu_score)
        print("Cam Score::", cam_score, "::Imu Score::", imu_score)
        print("Frame Number::", frameNum, "Total Score", tot_score)
        if(len(x)>100):
            x.popleft()
            y.popleft()
            z.popleft()
        
    else:
        cam_flag  = [0, 0, 0 , 0, 0]
        cam_score = [0, 0 ,0 , 0, 0]
        imu_flag  = 0
        imu_score = 0

    cv2.imshow('frame', inp_image)
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break
    frameNum = frameNum + 1
    
cv2.destroyAllWindows()
hf.close()

In [ ]:
def loadAndTrain():
    temp = []
    trainX, trainY = getDataset(np.array(t_imu), np.array([np.array(acc0)*2.0/32768.0, np.array(gyro0)*250.0/32768.0, np.array(acc1)*2.0/32768.0, np.array(gyro1)*250.0/32768.0]).reshape((len(t_imu),12)), in_imu, stride=1, sigLen = 88, isfeature=True)
    new_clf, score = trainModel(trainX, trainY, func='gaussian')

    #trainX, trainY = getDataset(np.array(t_imu), np.array([np.array(acc0), np.array(gyro0), np.array(acc1), np.array(gyro1)]).reshape((len(t_imu),12)), in_imu, stride=1, sigLen = 25, isfeature=True)
    #new_clf, score = trainModel(trainX, trainY, func='gaussian')
    #for i in range(150):
    #    trainX, trainY = getDataset(np.array(t_imu), np.array([np.array(acc0), np.array(gyro0), np.array(acc1), np.array(gyro1)]).reshape((len(t_imu),12)), in_imu, stride=5, sigLen = i+5, isfeature=True)
    #    new_clf, score = trainModel(trainX, trainY, func='gaussian')
    #    temp.append(score)
    for i in range(len(acc0)-27):
        data = np.array([acc0[i:i+25]*2.0/32768.0, gyro0[i:i+25]*250.0/32768.0, acc1[i:i+25]*2.0/32768.0, gyro1[i:i+25]*250.0/32768.0]).reshape(25,12)
        inp  = np.array((stdev(data), mean(data))).reshape((1,24))
        out = (new_clf.predict(inp))
        temp.append(out)
        #print(i, " == ", trainY[i], " == ", out)

    print((sum(np.array(trainY)==np.array(temp[0:2125]).reshape(2125,))/len(trainY))*100)
    hf.close()

In [ ]:
def realtimeData():
    camera = cv2.VideoCapture(1)
    ret, image = camera.read()

    #Now threading1 runs regardless of user input
    threading1 = threading.Thread(target=background)
    threading1.daemon = True
    threading1.start()

    #Open serial port for reading data from Intel Curie IMU
    ser0 = serial.Serial('/dev/ttyACM0', 115200) # Establish the connection on a specific port 0
    ser1 = serial.Serial('/dev/ttyACM1', 115200) # Establish the connection on a specific port 1
    ser0.flush()
    ser1.flush()

    result = list()
    counter= -1200
    flag   = 0
    tempc  = []

    threshold  = 10
    intent_thres_imu = 10
    imu_data   = []
    pred_count = 0   
    try:
        while True:
            inp_image = image                           #save a image copy as image frame grab is running on a separate thread
            print(detectObject(inp_image, result))      #ret, image = camera.read()
            cv2.imshow('frame', inp_image)
            if(cv2.waitKey(1) & 0xFF == ord('q')):
                break

            if(detectIntent(flag, counter)):            #print("Intention Probably is to pick up the bottle")
                while True:                             #as of now continous loop but should be more adaptive and intuitive #take care of 'garbage data'
                    data0 = ser0.readline().rstrip()
                    data0 = data0.split()
                    data1 = ser1.readline().rstrip()
                    data1 = data1.split()
                    if (len(data0) == 6 & len(data1) == 6):
                        imu_data.append([int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5])])
                        #imu_data.append([(int(data0[0])*2.0)/ 32768.0, (int(data0[1])*2.0)/ 32768.0, (int(data0[2])*2.0)/ 32768.0, (int(data0[3])*250.0)/32768.0, (int(data0[4])*250.0)/32768.0, (int(data0[5])*250.0)/32768.0])
                        #imu_data.append([float(time.time()),  int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5])])

                    if(len(imu_data)>24):
                        predicted = clf.predict(np.array((stdev(np.array(imu_data[-25:])), mean(np.array(imu_data[-25:])))).reshape((1,24)))
                        if(predicted == 1):
                            sys.stdout.write(" " + str(pred_count))
                            sys.stdout.flush()
                            pred_count = pred_count + 1
                            tempc.append(pred_count)
                        else:
                            sys.stdout.write(" " + str(pred_count))
                            sys.stdout.flush()
                            pred_count = pred_count - 1
                            tempc.append(pred_count)

    except KeyboardInterrupt:
        camera.release()
        cv2.destroyAllWindows()
        ser0.close()
        ser1.close()
        pass

In [ ]:
#pca = PCA(n_components=1, copy=True, whiten=True)
#transformed_dataset = PCA.fit_transform(pca, np.array(imu_data[-30:]))
#predicted = clf.predict(transformed_dataset.T)

#bucket    = get_buckets(np.array(imu_data[-30:]), 1, 30, 40)   #send the last 30 elements data to get_samples function
#predicted = clf.predict(np.reshape(bucket, (1, 40)))           #reshaping and sending for classification

#a, b = plotDataset(1,3000)
#x,y = getDataset(0, a[1:13], b)

#print(joblib.load('weights/1s_6sps.pkl').predict(np.array(x[4000]).reshape((1,24))))